In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import GPT2Tokenizer, GPT2Model

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2Model.from_pretrained('gpt2')
text = "The Fox is being late for work. The Cat is drinking the coffee. Who is more chill?"
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]